In [1]:
import pandas as pd
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
df = None
folderpath = "../../Data/QMEF/ILIAS_Elon_Musk_Tweets/"

In [3]:
# Some tweets are longer than one line. The line break character \n gets misinterpreted as new tweet. Replace all \n with white space but the first one.
for filename in os.listdir(folderpath):
    with open(os.path.join(folderpath,filename), 'r', encoding='latin1') as file:
        before, first, after = file.read().partition('\n')
        result = before + first + after.replace('\n', ' ')
    with open(os.path.join(folderpath,filename), 'w', encoding='latin1') as file:
        file.write(result)

In [4]:
# Interate through all tweets and append them to a pandas DataFrame
# Could use further optimization - Runtime 10 seconds.
for filename in os.listdir(folderpath):
    df = pd.concat([df, pd.read_csv(os.path.join(folderpath,filename), encoding='utf8', sep='\t')], axis=0)

In [5]:
# Correcting the Data Types
df['UTC'] = pd.to_datetime(df['UTC'])
df['CreatedAt'] = pd.to_datetime(df['CreatedAt'])
df.dtypes

TweetId                     int64
Text                       object
Name                       object
ScreenName                 object
UTC           datetime64[ns, UTC]
CreatedAt     datetime64[ns, UTC]
Favorites                   int64
Retweets                    int64
Language                   object
Client                     object
TweetType                  object
URLs                       object
Hashtags                    int64
Mentions                    int64
MediaType                  object
MediaURLs                  object
dtype: object

In [13]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    vs = analyzer.polarity_scores(text)
    return [vs['neg'], vs['pos'], vs['neu'], vs['compound']]

df[['neg', 'pos', 'neu', 'compund']] = df.apply(lambda x: get_sentiment(str(x['Text'])),result_type='expand', axis=1)
# TypeError: 'int' object is not iterable

In [14]:
df

,TweetId,Text,Name,ScreenName,UTC,CreatedAt,Favorites,Retweets,Language,Client,TweetType,URLs,Hashtags,Mentions,MediaType,MediaURLs,neg,pos,neu,compund
0,1321513255332941824,@MachinePix Little crane: “Am I just a toy to ...,Elon Musk,elonmusk,2020-10-28 18:04:47+00:00,2020-10-28 18:04:47+00:00,1762,52,en,"<a href=""http://twitter.com/download/iphone"" r...",Reply,NaN,0,1,NaN,,0.0,0.000,1.000,0.0000
0,1331745833629470722,@EvaFoxU We will have special colors for new R...,Elon Musk,elonmusk,2020-11-25 23:45:24+00:00,2020-11-25 23:45:24+00:00,3820,143,en,"<a href=""http://twitter.com/download/iphone"" r...",Reply,NaN,0,1,NaN,,0.0,0.278,0.722,0.6124
0,1349459692515434502,@NASASpaceflight Sea shanty tiktok takes it to...,Elon Musk,elonmusk,2021-01-13 20:53:57+00:00,2021-01-13 20:53:57+00:00,4237,140,en,"<a href=""http://twitter.com/download/iphone"" r...",Reply,NaN,0,1,NaN,,0.0,0.000,1.000,0.0000
0,1390920049372897281,@ericrihlmann @Tesla @nbcsnl @TimesSquareNYC @...,Elon Musk,elonmusk,2021-05-08 06:42:36+00:00,2021-05-08 06:42:36+00:00,13486,480,en,"<a href=""http://twitter.com/download/iphone"" r...",Reply,NaN,0,5,NaN,,0.0,0.406,0.594,0.6249
0,1390839584225955847,Cybertruck prototype in New York this weekend,Elon Musk,elonmusk,2021-05-08 01:22:52+00:00,2021-05-08 01:22:52+00:00,272338,16591,en,"<a href=""http://twitter.com/download/iphone"" r...",Tweet,NaN,0,0,NaN,,0.0,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1391164130204192771,@flcnhvy Might still need a VPN in some countries,Elon Musk,elonmusk,2021-05-08 22:52:29+00:00,2021-05-08 22:52:29+00:00,11331,659,en,"<a href=""http://twitter.com/download/iphone"" r...",Reply,NaN,0,1,NaN,,0.0,0.000,1.000,0.0000
0,1391130755787599881,Special link to view SNL outside USA https://t...,Elon Musk,elonmusk,2021-05-08 20:39:52+00:00,2021-05-08 20:39:52+00:00,175011,27144,en,"<a href=""http://twitter.com/download/iphone"" r...",Tweet,https://m.youtube.com/watch?v=VJX9ymv_vUQ&feat...,0,0,NaN,,0.0,0.278,0.722,0.4019
0,1390955387654914052,First time a Falcon rocket booster will reach ...,Elon Musk,elonmusk,2021-05-08 09:03:01+00:00,2021-05-08 09:03:01+00:00,95974,6651,en,"<a href=""http://twitter.com/download/iphone"" r...",Tweet,https://twitter.com/spacex/status/139084323233...,0,0,NaN,,0.0,0.084,0.916,0.0258
0,1390929314355548165,@WholeMarsBlog Yup 🤣🤣,Elon Musk,elonmusk,2021-05-08 07:19:25+00:00,2021-05-08 07:19:25+00:00,9159,538,und,"<a href=""http://twitter.com/download/iphone"" r...",Reply,NaN,0,1,NaN,,0.0,0.390,0.610,0.7506
